In [151]:
import numpy
import pandas

from fisher import pvalue

Take mutations data set and get all rows with UNIQUEIDs that also contain CMs, as defined by our (filtered) hit list

In [142]:
MUTATIONS = pandas.read_pickle('tb_rnap_compensation/tables/MUTATIONS.pkl.gz')
MUTATIONS.reset_index(inplace=True)
MUTATIONS = MUTATIONS[(MUTATIONS.IS_FILTER_PASS) & (~MUTATIONS.IS_HET) & (~MUTATIONS.IS_NULL) & (~MUTATIONS.MUTATION.str[-1].isin(['O','X']))]
MUTATIONS = MUTATIONS.astype({'GENE':'str'})
MUTATIONS['GENE_MUTATION'] =MUTATIONS['GENE'] + '_' +MUTATIONS['MUTATION']
MUTATIONS

,UNIQUEID,GENE,MUTATION,POSITION,AMINO_ACID_NUMBER,GENOME_INDEX,NUCLEOTIDE_NUMBER,REF,ALT,IS_SNP,...,IS_NULL,IS_FILTER_PASS,ELEMENT_TYPE,MUTATION_TYPE,INDEL_LENGTH,INDEL_1,INDEL_2,SITEID,NUMBER_NUCLEOTIDE_CHANGES,GENE_MUTATION
0,site.02.subj.0958.lab.22A197.iso.1,rpoB,P45S,45.0,45.0,NaN,NaN,ccg,tcg,True,...,False,True,GENE,AAM,NaN,,,02,1,rpoB_P45S
1,site.02.subj.0958.lab.22A197.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,...,False,True,GENE,AAM,NaN,,,02,1,rpoB_S450L
2,site.02.subj.0958.lab.22A197.iso.1,rpoB,A1075A,1075.0,1075.0,NaN,NaN,gct,gcc,True,...,False,True,GENE,AAM,NaN,,,02,1,rpoB_A1075A
3,site.02.subj.0958.lab.22A197.iso.1,rpoC,D271E,271.0,271.0,NaN,NaN,gac,gag,True,...,False,True,GENE,AAM,NaN,,,02,1,rpoC_D271E
6,site.02.subj.0359.lab.222018-14.iso.1,rpoB,A1075A,1075.0,1075.0,NaN,NaN,gct,gcc,True,...,False,True,GENE,AAM,NaN,,,02,1,rpoB_A1075A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517506,site.10.subj.YA00023171.lab.YA00023171.iso.1,rpoC,A542A,542.0,542.0,NaN,NaN,gcc,gcg,True,...,False,True,GENE,AAM,NaN,,,10,1,rpoC_A542A
517507,site.10.subj.SADH00634109_S16.lab.DH00634109_S...,rpoC,A542A,542.0,542.0,NaN,NaN,gcc,gcg,True,...,False,True,GENE,AAM,NaN,,,10,1,rpoC_A542A
517508,site.10.subj.YA00044774.lab.YA00044774.iso.1,rpoC,A542A,542.0,542.0,NaN,NaN,gcc,gcg,True,...,False,True,GENE,AAM,NaN,,,10,1,rpoC_A542A
517509,site.10.subj.YA00166043.lab.YA00166043.iso.1,rpoB,D435V,435.0,435.0,NaN,NaN,gac,gtc,True,...,False,True,GENE,AAM,NaN,,,10,1,rpoB_D435V


In [141]:
hits = pandas.read_excel('hits_filtered.xlsx') #for hits_filtered also no overlaps of samples with >1 nucleotide changes in resistance gene and CMs
hits

,resistant_mutation,other_mutation,prevalence
0,rpoB_S450L,rpoC_N826T,64
1,rpoB_S450L,rpoC_D485Y,194
2,rpoB_S450L,rpoB_A692T,79
3,rpoB_S450L,rpoB_I480V,78
4,rpoB_S450L,rpoA_V183G,77
5,rpoB_S450L,rpoC_E1092D,1989
6,rpoB_S450L,rpoB_A286V,56
7,rpoB_S450L,rpoB_V496A,60
8,rpoB_S450L,rpoC_T812I,51
9,rpoB_S450L,rpoC_G519D,51


In [115]:
CM_samples = MUTATIONS[(MUTATIONS.GENE_MUTATION.isin(hits.other_mutation.values.tolist()))].UNIQUEID
CM_samples

0                        site.02.subj.0958.lab.22A197.iso.1
22                   site.02.subj.0739.lab.2013221518.iso.1
26                    site.02.subj.0370.lab.222044-14.iso.1
44                       site.02.subj.0885.lab.22A119.iso.1
86                    site.02.subj.0411.lab.235087-14.iso.1
                                ...                        
517466    site.10.subj.SATRL0116187_S19.lab.TRL0116187_S...
517470    site.10.subj.SADG00611914_S7.lab.DG00611914_S7...
517489    site.10.subj.SATRL0115669_S18.lab.TRL0115669_S...
517500         site.10.subj.YA00008913.lab.YA00008913.iso.1
517505         site.10.subj.YA00008913.lab.YA00008913.iso.1
Name: UNIQUEID, Length: 21665, dtype: object

Get all mutations from this subset of samples with CMs that have more than 1 mutation in a codon

In [116]:
MUTATIONS[(MUTATIONS.UNIQUEID.isin(CM_samples)) & (MUTATIONS.NUMBER_NUCLEOTIDE_CHANGES > 1)]

,UNIQUEID,GENE,MUTATION,POSITION,AMINO_ACID_NUMBER,GENOME_INDEX,NUCLEOTIDE_NUMBER,REF,ALT,IS_SNP,...,IS_NULL,IS_FILTER_PASS,ELEMENT_TYPE,MUTATION_TYPE,INDEL_LENGTH,INDEL_1,INDEL_2,SITEID,NUMBER_NUCLEOTIDE_CHANGES,GENE_MUTATION
37417,site.24.subj.PT-414.lab.2014-094.iso.1,rpoC,L789A,789.0,789.0,NaN,NaN,ttg,gcg,True,...,False,True,GENE,AAM,NaN,,,24,2,rpoC_L789A
40164,site.11.subj.MDR077.lab.MYCOBACTERIUM_TUBERCUL...,rpoC,L789A,789.0,789.0,NaN,NaN,ttg,gcg,True,...,False,True,GENE,AAM,NaN,,,11,2,rpoC_L789A
46879,site.04.subj.04686.lab.ADI8625.iso.1,rpoB,H445C,445.0,445.0,NaN,NaN,cac,tgc,True,...,False,True,GENE,AAM,NaN,,,04,2,rpoB_H445C
48015,site.04.subj.01440.lab.726455.iso.1,rpoB,S450F,450.0,450.0,NaN,NaN,tcg,ttt,True,...,False,True,GENE,AAM,NaN,,,04,2,rpoB_S450F
51935,site.04.subj.04316.lab.900539.iso.1,rpoC,A1044L,1044.0,1044.0,NaN,NaN,gcg,ctg,True,...,False,True,GENE,AAM,NaN,,,04,2,rpoC_A1044L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510597,site.28.subj.1037.lab.R24968.iso.1,rpoC,A1299T,1299.0,1299.0,NaN,NaN,gcg,acc,True,...,False,True,GENE,AAM,NaN,,,28,2,rpoC_A1299T
510601,site.28.subj.1037.lab.R24968.iso.1,rpoC,S1313S,1313.0,1313.0,NaN,NaN,agc,tcc,True,...,False,True,GENE,AAM,NaN,,,28,2,rpoC_S1313S
510617,site.28.subj.1037.lab.R24968.iso.1,sigA,S433S,433.0,433.0,NaN,NaN,agc,tcg,True,...,False,True,GENE,AAM,NaN,,,28,3,sigA_S433S
510633,site.28.subj.1037.lab.R24968.iso.1,sigA,L477L,477.0,477.0,NaN,NaN,cta,ttg,True,...,False,True,GENE,AAM,NaN,,,28,2,sigA_L477L


Do the same for the original dataset

In [117]:
revertants_all = MUTATIONS[MUTATIONS.NUMBER_NUCLEOTIDE_CHANGES > 1]
revertants_all

,UNIQUEID,GENE,MUTATION,POSITION,AMINO_ACID_NUMBER,GENOME_INDEX,NUCLEOTIDE_NUMBER,REF,ALT,IS_SNP,...,IS_NULL,IS_FILTER_PASS,ELEMENT_TYPE,MUTATION_TYPE,INDEL_LENGTH,INDEL_1,INDEL_2,SITEID,NUMBER_NUCLEOTIDE_CHANGES,GENE_MUTATION
817,site.02.subj.0926.lab.22A161.iso.1,rpoB,S450F,450.0,450.0,NaN,NaN,tcg,ttc,True,...,False,True,GENE,AAM,NaN,,,02,2,rpoB_S450F
859,site.02.subj.0893.lab.22A127.iso.1,rpoB,D435F,435.0,435.0,NaN,NaN,gac,ttc,True,...,False,True,GENE,AAM,NaN,,,02,2,rpoB_D435F
1022,site.02.subj.0197.lab.2013221241.iso.1,sigA,A55S,55.0,55.0,NaN,NaN,gcc,tcg,True,...,False,True,GENE,AAM,NaN,,,02,2,sigA_A55S
1506,site.02.subj.0074.lab.22A026.iso.1,rpoB,H445C,445.0,445.0,NaN,NaN,cac,tgc,True,...,False,True,GENE,AAM,NaN,,,02,2,rpoB_H445C
2244,site.05.subj.LR-2335.lab.FN-01418-18.iso.1,rpoB,S450M,450.0,450.0,NaN,NaN,tcg,atg,True,...,False,True,GENE,AAM,NaN,,,05,2,rpoB_S450M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515444,site.10.subj.YA00134694.lab.YA00134694.iso.1,rpoB,H445G,445.0,445.0,NaN,NaN,cac,ggc,True,...,False,True,GENE,AAM,NaN,,,10,2,rpoB_H445G
516308,site.10.subj.YA00128989.lab.YA00128989.iso.1,rpoB,S441V,441.0,441.0,NaN,NaN,tcg,gtg,True,...,False,True,GENE,AAM,NaN,,,10,2,rpoB_S441V
516789,site.10.subj.YA00022288.lab.YA00022288.iso.1,rpoB,H445G,445.0,445.0,NaN,NaN,cac,ggc,True,...,False,True,GENE,AAM,NaN,,,10,2,rpoB_H445G
517411,site.10.subj.LA00835602.lab.LA00835602.iso.1,rpoB,S450Q,450.0,450.0,NaN,NaN,tcg,cag,True,...,False,True,GENE,AAM,NaN,,,10,2,rpoB_S450Q


Now only look at those samples where we see > 1 mutation in a resistance codon

In [118]:
EFFECTS = pandas.read_pickle('tb_rnap_compensation/tables/EFFECTS.pkl.gz')
EFFECTS.reset_index(inplace=True)
EFFECTS = EFFECTS.astype({'GENE':'str'})
EFFECTS['GENE_MUTATION'] = EFFECTS['GENE'] + '_' + EFFECTS['MUTATION']
EFFECTS
resistance_codons = EFFECTS[(EFFECTS.PREDICTION=='R') & (~EFFECTS.MUTATION.str[-1].isin(['O','X']))].GENE_MUTATION.str[0:9]
resistance_codons

3          rpoB_S450
16         rpoB_D435
45         rpoB_S450
51         rpoB_S450
89         rpoB_S450
             ...    
1074634    rpoB_S450
1074647    rpoB_S450
1074653    rpoB_S450
1074661    rpoB_S450
1074680    rpoB_D435
Name: GENE_MUTATION, Length: 20829, dtype: object

In [119]:
revertants = revertants_all[revertants_all.GENE_MUTATION.str[0:9].isin(resistance_codons)]
revertants

,UNIQUEID,GENE,MUTATION,POSITION,AMINO_ACID_NUMBER,GENOME_INDEX,NUCLEOTIDE_NUMBER,REF,ALT,IS_SNP,...,IS_NULL,IS_FILTER_PASS,ELEMENT_TYPE,MUTATION_TYPE,INDEL_LENGTH,INDEL_1,INDEL_2,SITEID,NUMBER_NUCLEOTIDE_CHANGES,GENE_MUTATION
817,site.02.subj.0926.lab.22A161.iso.1,rpoB,S450F,450.0,450.0,NaN,NaN,tcg,ttc,True,...,False,True,GENE,AAM,NaN,,,02,2,rpoB_S450F
859,site.02.subj.0893.lab.22A127.iso.1,rpoB,D435F,435.0,435.0,NaN,NaN,gac,ttc,True,...,False,True,GENE,AAM,NaN,,,02,2,rpoB_D435F
1506,site.02.subj.0074.lab.22A026.iso.1,rpoB,H445C,445.0,445.0,NaN,NaN,cac,tgc,True,...,False,True,GENE,AAM,NaN,,,02,2,rpoB_H445C
2244,site.05.subj.LR-2335.lab.FN-01418-18.iso.1,rpoB,S450M,450.0,450.0,NaN,NaN,tcg,atg,True,...,False,True,GENE,AAM,NaN,,,05,2,rpoB_S450M
2518,site.05.subj.CA-0117.lab.CO-07529-18.iso.1,rpoB,S450F,450.0,450.0,NaN,NaN,tcg,ttt,True,...,False,True,GENE,AAM,NaN,,,05,2,rpoB_S450F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515444,site.10.subj.YA00134694.lab.YA00134694.iso.1,rpoB,H445G,445.0,445.0,NaN,NaN,cac,ggc,True,...,False,True,GENE,AAM,NaN,,,10,2,rpoB_H445G
516308,site.10.subj.YA00128989.lab.YA00128989.iso.1,rpoB,S441V,441.0,441.0,NaN,NaN,tcg,gtg,True,...,False,True,GENE,AAM,NaN,,,10,2,rpoB_S441V
516789,site.10.subj.YA00022288.lab.YA00022288.iso.1,rpoB,H445G,445.0,445.0,NaN,NaN,cac,ggc,True,...,False,True,GENE,AAM,NaN,,,10,2,rpoB_H445G
517411,site.10.subj.LA00835602.lab.LA00835602.iso.1,rpoB,S450Q,450.0,450.0,NaN,NaN,tcg,cag,True,...,False,True,GENE,AAM,NaN,,,10,2,rpoB_S450Q


In [122]:
CM_and_reversion = revertants[revertants.UNIQUEID.isin(CM_samples)]
CM_and_reversion

,UNIQUEID,GENE,MUTATION,POSITION,AMINO_ACID_NUMBER,GENOME_INDEX,NUCLEOTIDE_NUMBER,REF,ALT,IS_SNP,...,IS_NULL,IS_FILTER_PASS,ELEMENT_TYPE,MUTATION_TYPE,INDEL_LENGTH,INDEL_1,INDEL_2,SITEID,NUMBER_NUCLEOTIDE_CHANGES,GENE_MUTATION
46879,site.04.subj.04686.lab.ADI8625.iso.1,rpoB,H445C,445.0,445.0,NaN,NaN,cac,tgc,True,...,False,True,GENE,AAM,NaN,,,04,2,rpoB_H445C
48015,site.04.subj.01440.lab.726455.iso.1,rpoB,S450F,450.0,450.0,NaN,NaN,tcg,ttt,True,...,False,True,GENE,AAM,NaN,,,04,2,rpoB_S450F
68900,site.04.subj.00712.lab.708511.iso.1,rpoB,H445S,445.0,445.0,NaN,NaN,cac,tcc,True,...,False,True,GENE,AAM,NaN,,,04,2,rpoB_H445S
77590,site.04.subj.00779.lab.704350.iso.1,rpoB,I491Y,491.0,491.0,NaN,NaN,atc,tac,True,...,False,True,GENE,AAM,NaN,,,04,2,rpoB_I491Y
87586,site.04.subj.01578.lab.728774.iso.1,rpoB,S450C,450.0,450.0,NaN,NaN,tcg,tgt,True,...,False,True,GENE,AAM,NaN,,,04,2,rpoB_S450C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469241,site.06.subj.DJB47.lab.06MIL1523.iso.1,rpoB,H445G,445.0,445.0,NaN,NaN,cac,ggc,True,...,False,True,GENE,AAM,NaN,,,06,2,rpoB_H445G
476262,site.06.subj.D006517.lab.06MIL1345.iso.1,rpoB,R448K,448.0,448.0,NaN,NaN,cga,aaa,True,...,False,True,GENE,AAM,NaN,,,06,2,rpoB_R448K
488234,site.06.subj.MHL_0181-14.lab.06MIL0203.iso.1,rpoB,D435F,435.0,435.0,NaN,NaN,gac,ttc,True,...,False,True,GENE,AAM,NaN,,,06,2,rpoB_D435F
509988,site.28.subj.1037.lab.R24968.iso.1,rpoB,S431S,431.0,431.0,NaN,NaN,agc,tcg,True,...,False,True,GENE,AAM,NaN,,,28,3,rpoB_S431S


Compare number of samples with CMs against those with reversions

In [124]:
print(len(CM_and_reversion.UNIQUEID.unique()))
print(len(revertants.UNIQUEID.unique()))

92
671


Do fishers test 

In [153]:
#create contingency table: top: reversion, side: CM
#     F         T
#
# F  42.478     579
#
# T  21.573     92
p = pvalue(42478, 579, 21573, 92)
p

PValues(left_tail=6.354e-32, right_tail=1, two_tail=5.763e-06)

Look at type of CM and resistance mutations among 92 samples with reversion

In [150]:
MUTATIONS[MUTATIONS.UNIQUEID.isin(CM_and_reversion.UNIQUEID.unique())]

,UNIQUEID,GENE,MUTATION,POSITION,AMINO_ACID_NUMBER,GENOME_INDEX,NUCLEOTIDE_NUMBER,REF,ALT,IS_SNP,...,IS_NULL,IS_FILTER_PASS,ELEMENT_TYPE,MUTATION_TYPE,INDEL_LENGTH,INDEL_1,INDEL_2,SITEID,NUMBER_NUCLEOTIDE_CHANGES,GENE_MUTATION
46879,site.04.subj.04686.lab.ADI8625.iso.1,rpoB,H445C,445.0,445.0,NaN,NaN,cac,tgc,True,...,False,True,GENE,AAM,NaN,,,04,2,rpoB_H445C
46880,site.04.subj.04686.lab.ADI8625.iso.1,rpoB,G876G,876.0,876.0,NaN,NaN,ggt,ggg,True,...,False,True,GENE,AAM,NaN,,,04,1,rpoB_G876G
46881,site.04.subj.04686.lab.ADI8625.iso.1,rpoB,A1075A,1075.0,1075.0,NaN,NaN,gct,gcc,True,...,False,True,GENE,AAM,NaN,,,04,1,rpoB_A1075A
46882,site.04.subj.04686.lab.ADI8625.iso.1,rpoB,c-61t,-61.0,NaN,759746.0,-61.0,c,t,True,...,False,True,GENE,SNP,NaN,,,04,0,rpoB_c-61t
48015,site.04.subj.01440.lab.726455.iso.1,rpoB,S450F,450.0,450.0,NaN,NaN,tcg,ttt,True,...,False,True,GENE,AAM,NaN,,,04,2,rpoB_S450F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510645,site.28.subj.1037.lab.R24968.iso.1,sigA,L514L,514.0,514.0,NaN,NaN,ttg,ctg,True,...,False,True,GENE,AAM,NaN,,,28,1,sigA_L514L
510646,site.28.subj.1037.lab.R24968.iso.1,sigA,H516H,516.0,516.0,NaN,NaN,cat,cac,True,...,False,True,GENE,AAM,NaN,,,28,1,sigA_H516H
510647,site.28.subj.1037.lab.R24968.iso.1,sigA,P517P,517.0,517.0,NaN,NaN,ccg,ccc,True,...,False,True,GENE,AAM,NaN,,,28,1,sigA_P517P
510648,site.28.subj.1037.lab.R24968.iso.1,sigA,S520S,520.0,520.0,NaN,NaN,tca,tcg,True,...,False,True,GENE,AAM,NaN,,,28,1,sigA_S520S
